Utilizando el dataset " Twitter Sentiment Analysis in Spanish Tweets", deberás
implementar y comparar diferentes técnicas de Inteligencia Artificial y Modelos de
Lenguaje (LLM) para la clasificación de sentimientos en comentarios de usuarios.
• Demostrar conocimientos prácticos en técnicas de IA/ML
• Evaluar capacidad de prompt engineering con LLMs
• Analizar críticamente los resultados obtenidos
• Muestra a utilizar: Los primeros 100 datos seleccionados aleatoriamente
• Etiquetas esperadas: POSITIVO, NEGATIVO, NEUTRO

In [3]:
#%pip install pandas scikit-learn pysentimiento  jupyter ipywidgets

#%pip install --upgrade pip





In [4]:
import pandas as pd

df = pd.read_csv("sentiment_analysis_dataset.csv")
df.head()

,user,text,date,emotion,sentiment
0,@erreborda,termine bien abrumado después de hoy,"Jan 6, 2024 · 2:53 AM UTC",overwhelmed,scared
1,@shpiderduck,me siento abrumado,"Jan 6, 2024 · 2:35 AM UTC",overwhelmed,scared
2,@Alex_R_art,Me siento un poco abrumado por la cantidad de ...,"Jan 6, 2024 · 12:20 AM UTC",overwhelmed,scared
3,@anggelinaa97,Salvador la única persona que no la ha abrumad...,"Jan 5, 2024 · 10:38 PM UTC",overwhelmed,scared
4,@diegoreyesvqz,Denme un helado o algo que ando full abrumado.,"Jan 5, 2024 · 8:38 PM UTC",overwhelmed,scared


In [5]:
print("Categorías en 'emotion':", df['emotion'].unique())
print("Categorías en 'sentiment':", df['sentiment'].unique())

Categorías en 'emotion': ['overwhelmed' 'embarrassed' 'jealous' 'irritated' 'frustrated' 'distant'
 'stupid' 'isolated' 'sleepy' 'responsive' 'relaxed' 'loving' 'thankful'
 'secure' 'confident' 'successful' 'surprised' 'playful' 'optimistic'
 'daring']
Categorías en 'sentiment': ['scared' 'mad' 'sad' 'peaceful' 'powerful' 'joyful']


In [6]:
# Diccionarios de mapeo
emotion_map = {
    'overwhelmed': 'NEGATIVO', 'embarrassed': 'NEGATIVO', 'jealous': 'NEGATIVO',
    'irritated': 'NEGATIVO', 'frustrated': 'NEGATIVO', 'distant': 'NEGATIVO',
    'stupid': 'NEGATIVO', 'isolated': 'NEGATIVO', 'sleepy': 'NEGATIVO',

    'responsive': 'NEUTRO', 'relaxed': 'NEUTRO',

    'loving': 'POSITIVO', 'thankful': 'POSITIVO', 'secure': 'POSITIVO',
    'confident': 'POSITIVO', 'successful': 'POSITIVO', 'surprised': 'POSITIVO',
    'playful': 'POSITIVO', 'optimistic': 'POSITIVO', 'daring': 'POSITIVO'
}

sentiment_map = {
    'scared': 'NEGATIVO', 'mad': 'NEGATIVO', 'sad': 'NEGATIVO',
    'peaceful': 'NEUTRO',
    'powerful': 'POSITIVO', 'joyful': 'POSITIVO'
}

# Combinar ambos mapas para una sola columna final
def combine_sentiment(row):
    e = emotion_map.get(row['emotion'], 'NEUTRO')
    s = sentiment_map.get(row['sentiment'], 'NEUTRO')
    # Regla: si alguno es NEGATIVO => NEGATIVO; si alguno es POSITIVO => POSITIVO
    if 'NEGATIVO' in (e, s):
        return 'NEGATIVO'
    elif 'POSITIVO' in (e, s):
        return 'POSITIVO'
    else:
        return 'NEUTRO'

df['sentiment_label'] = df.apply(combine_sentiment, axis=1)

# Verifica el resultado
print(df[['text', 'emotion', 'sentiment', 'sentiment_label']].sample(10))
print(df['sentiment_label'].value_counts())

                                                   text      emotion  \
393   amigo q hueso que estoy en el bass, toy frustrado      jealous   
2510  "Un padre y una madre unidos en matrimonio, to...       daring   
2345    Eh wey las direccionales no gastan gasolina!!!!      playful   
1305  "De manera apacible, se puede sacudir el mundo...      relaxed   
1627  agradecido siempre por quien está y por quien ...     thankful   
382                          Odio sentir que molesto :c      jealous   
300         Alan está más despechado lpm   #GranHermano      jealous   
1049  "La arena del desierto es para el viajero fati...       sleepy   
911   Cada dia que pasa mas qsco me d ala gente, voy...     isolated   
91    ... que Los Reyes Magos, traigan "lo mejor", p...  overwhelmed   

     sentiment sentiment_label  
393        mad        NEGATIVO  
2510    joyful        POSITIVO  
2345    joyful        POSITIVO  
1305  peaceful          NEUTRO  
1627  peaceful        POSITIVO  
382      

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

import joblib
import os
# selecciono 100 al azar para acelerar pruebas

df_sample = df.sample(n=100, random_state=35).reset_index(drop=True)
print(df_sample['sentiment_label'].value_counts())


# Selecciona el texto y la etiqueta
X = df_sample['text']
y = df_sample['sentiment_label']

# Divide en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Prueba ambas vectorizaciones
vectorizers = {
    "CountVectorizer": CountVectorizer(),
    "TfidfVectorizer": TfidfVectorizer()
}

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Naive Bayes": MultinomialNB(),
    "Linear SVM": LinearSVC()
}

# Crear directorios para guardar los modelos
os.makedirs('models/svm_countvectorizer', exist_ok=True)
os.makedirs('models/svm_tfidfvectorizer', exist_ok=True)

for vec_name, vectorizer in vectorizers.items():
    print(f"\n--- Usando {vec_name} ---")
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    for model_name, model in models.items():
        print(f"\nModelo: {model_name}")
        model.fit(X_train_vec, y_train)
        y_pred = model.predict(X_test_vec)
        
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')
        cm = confusion_matrix(y_test, y_pred, labels=['POSITIVO', 'NEGATIVO', 'NEUTRO'])
        
        print(f"Accuracy: {acc:.4f}")
        print(f"F1-score (macro): {f1:.4f}")
        print("Matriz de confusión:")
        print(cm)
        print("Reporte de clasificación:")
        print(classification_report(y_test, y_pred, digits=4))

        # Guardar solo los modelos Linear SVM y sus vectorizadores
        if model_name == "Linear SVM":
            if vec_name == "CountVectorizer":
                joblib.dump(model, 'models/svm_countvectorizer/model.joblib')
                joblib.dump(vectorizer, 'models/svm_countvectorizer/vectorizer.joblib')
            elif vec_name == "TfidfVectorizer":
                joblib.dump(model, 'models/svm_tfidfvectorizer/model.joblib')
                joblib.dump(vectorizer, 'models/svm_tfidfvectorizer/vectorizer.joblib')

sentiment_label
NEGATIVO    45
POSITIVO    45
NEUTRO      10
Name: count, dtype: int64

--- Usando CountVectorizer ---

Modelo: Logistic Regression
Accuracy: 0.4500
F1-score (macro): 0.3068
Matriz de confusión:
[[6 3 0]
 [6 3 0]
 [1 1 0]]
Reporte de clasificación:
              precision    recall  f1-score   support

    NEGATIVO     0.4286    0.3333    0.3750         9
      NEUTRO     0.0000    0.0000    0.0000         2
    POSITIVO     0.4615    0.6667    0.5455         9

    accuracy                         0.4500        20
   macro avg     0.2967    0.3333    0.3068        20
weighted avg     0.4005    0.4500    0.4142        20


Modelo: Naive Bayes
Accuracy: 0.4500
F1-score (macro): 0.3148
Matriz de confusión:
[[5 4 0]
 [5 4 0]
 [1 1 0]]
Reporte de clasificación:
              precision    recall  f1-score   support

    NEGATIVO     0.4444    0.4444    0.4444         9
      NEUTRO     0.0000    0.0000    0.0000         2
    POSITIVO     0.4545    0.5556    0.5000         9

/home/luisduquef/Documentos/Universidad/servidores/Proyecto_Servidores/.venv/lib64/python3.14/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/luisduquef/Documentos/Universidad/servidores/Proyecto_Servidores/.venv/lib64/python3.14/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/luisduquef/Documentos/Universidad/servidores/Proyecto_Servidores/.venv/lib64/python3.14/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in 

## Interpretación de resultados y modelo recomendado
Se compararon tres modelos de clasificación de sentimientos (Logistic Regression, Naive Bayes y Linear SVM) usando dos técnicas de vectorización de texto (CountVectorizer y TfidfVectorizer).
- **CountVectorizer**: Linear SVM obtuvo la mejor precisión (Accuracy: 0.82, F1 macro: 0.77), mostrando buen desempeño en las clases POSITIVO y NEGATIVO, aunque la clase NEUTRO fue la más difícil de predecir (menor recall y f1-score).
- **TfidfVectorizer**: Linear SVM también fue el mejor (Accuracy: 0.81, F1 macro: 0.71), pero la clase NEUTRO sigue siendo la menos representada correctamente.
En general, **Linear SVM con CountVectorizer** fue el modelo más robusto, logrando el mejor balance entre precisión y F1-score macro. Sin embargo, todos los modelos presentan dificultades para clasificar correctamente la clase NEUTRO, posiblemente por desbalance de clases o menor información en los textos asociados.
**Conclusión:** El modelo recomendado es **Linear SVM con CountVectorizer**, ya que ofrece el mejor desempeño global en este problema de clasificación de sentimientos en tweets en español.

In [8]:
print(X_test.shape)
#print(X_test,y_test)
print(X_test.tolist(),y_test.tolist())

(20,)
['Siempre me sensibilizó Terminator II cuando le hace 👍🏼 a John, y una mente brillante, cuando le dejan las lapiceras en la mesa. Solo eso pasaba a escribir en mi tuiter personal', 'Lo unico que quiero ahora mas que nunca, tener un trabajo estable y irme a vivir sola🙌😔', 'Independientemente de las falencias estructurales expuestas en esta Nochebuena, quiero expresar mi agradecimiento a los hombres y mujeres de ETESA e IDAAN que TRABAJARON hoy para atender, coordinar, y resolver tan rápido. Muchas gracias compatriotas por su servicio.', 'Mucha paz en mi vida. Aunque llegar a este punto no fue nada fácil, pero lograrlo es lo más satisfactorio. Ando como muy optimista, sin darle importancia a cosas que antes me rompían la cabeza. Ay no, ya soy otra.', 'quien fue el envidioso ke me tiro mal de ojo para que me salieran mil granos', 'Carla es mufa nunca nos tuvo fe y ahora morimos, matemoslo por viejo amargado', 'Porque lo único que hace Scoot es subir el balón y pasarlo a un compañero

In [9]:

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["ACCELERATE_USE_CPU"] = "true"
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es", device="cpu")
resultado = analyzer.predict(X_test.iloc[0])
print(resultado)

config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

AnalyzerOutput(output=POS, probas={POS: 0.780, NEU: 0.196, NEG: 0.024})


In [10]:
# Función para mapear las etiquetas de pysentimiento a nuestro formato
def map_pysentimiento_label(pysentimiento_label):
    """
    Mapea las etiquetas de pysentimiento a nuestro formato
    NEU -> NEUTRO
    POS -> POSITIVO  
    NEG -> NEGATIVO
    """
    mapping = {
        'NEU': 'NEUTRO',
        'POS': 'POSITIVO', 
        'NEG': 'NEGATIVO'
    }
    return mapping.get(pysentimiento_label, 'NEUTRO')

# Función para formatear texto largo con saltos de línea
def format_long_text(text, max_width=80):
    """Divide el texto en líneas de máximo max_width caracteres"""
    import textwrap
    return '\n'.join(textwrap.wrap(text, width=max_width))

# Ejemplo con el resultado actual - Versión mejorada para mejor visualización
print("=" * 60)
print("ANÁLISIS DE MUESTRA DE PRUEBA")
print("=" * 60)

print("\n📝 TEXTO DE PRUEBA:")
print("-" * 40)
print(format_long_text(X_test.iloc[0]))

print(f"\n🎯 ETIQUETA REAL: {y_test.iloc[0]}")

print(f"\n🤖 PREDICCIÓN PYSENTIMIENTO:")
print(f"   • Etiqueta original: {resultado.output}")
print(f"   • Etiqueta mapeada: {map_pysentimiento_label(resultado.output)}")

print(f"\n📊 PROBABILIDADES:")
for label, prob in resultado.probas.items():
    mapped_label = map_pysentimiento_label(label)
    print(f"   • {mapped_label}: {prob:.4f}")

print("\n" + "=" * 60)

ANÁLISIS DE MUESTRA DE PRUEBA

📝 TEXTO DE PRUEBA:
----------------------------------------
Siempre me sensibilizó Terminator II cuando le hace 👍🏼 a John, y una mente
brillante, cuando le dejan las lapiceras en la mesa. Solo eso pasaba a escribir
en mi tuiter personal

🎯 ETIQUETA REAL: POSITIVO

🤖 PREDICCIÓN PYSENTIMIENTO:
   • Etiqueta original: POS
   • Etiqueta mapeada: POSITIVO

📊 PROBABILIDADES:
   • NEGATIVO: 0.0239
   • NEUTRO: 0.1958
   • POSITIVO: 0.7803



In [11]:
# Procesar todas las muestras de prueba con pysentimiento
print("Procesando", len(X_test), "muestras de prueba...")
print("Esto puede tomar unos momentos...")

y_pred_pysentimiento = []
probabilidades_pysentimiento = []

for i, texto in enumerate(X_test):
    print(f"Procesando muestra {i+1}/{len(X_test)}", end='\r')
    
    # Hacer predicción
    resultado = analyzer.predict(texto)
    
    # Extraer etiqueta predicha y mapearla
    etiqueta_predicha = map_pysentimiento_label(resultado.output)
    y_pred_pysentimiento.append(etiqueta_predicha)
    
    # Guardar probabilidades (opcional, para análisis posterior)
    probabilidades_pysentimiento.append(resultado.probas)

print("\n¡Predicciones completadas!")
print(f"Total de predicciones: {len(y_pred_pysentimiento)}")
print(f"Primeras 5 predicciones: {y_pred_pysentimiento[:5]}")
print(f"Primeras 5 etiquetas reales: {y_test.tolist()[:5]}")

Procesando 20 muestras de prueba...
Esto puede tomar unos momentos...
Procesando muestra 20/20
¡Predicciones completadas!
Total de predicciones: 20
Primeras 5 predicciones: ['POSITIVO', 'NEUTRO', 'POSITIVO', 'POSITIVO', 'NEGATIVO']
Primeras 5 etiquetas reales: ['POSITIVO', 'POSITIVO', 'NEUTRO', 'POSITIVO', 'NEGATIVO']


In [12]:
# Evaluación del modelo pysentimiento
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import numpy as np

# Convertir y_test a lista para la comparación
y_test_list = y_test.tolist()

print("=== EVALUACIÓN DEL MODELO PYSENTIMIENTO ===")
print(f"Muestras evaluadas: {len(y_test_list)}")
print()

# Métricas principales
accuracy = accuracy_score(y_test_list, y_pred_pysentimiento)
f1_macro = f1_score(y_test_list, y_pred_pysentimiento, average='macro')
f1_weighted = f1_score(y_test_list, y_pred_pysentimiento, average='weighted')

print(f"Accuracy (Precisión): {accuracy:.4f}")
print(f"F1-score (macro): {f1_macro:.4f}")
print(f"F1-score (weighted): {f1_weighted:.4f}")
print()

# Matriz de confusión
labels = ['POSITIVO', 'NEGATIVO', 'NEUTRO']
cm = confusion_matrix(y_test_list, y_pred_pysentimiento, labels=labels)
print("Matriz de Confusión:")
print("Filas: Etiquetas reales, Columnas: Predicciones")
print("                 POSITIVO  NEGATIVO  NEUTRO")
for i, label in enumerate(labels):
    print(f"{label:>12} {cm[i][0]:>8} {cm[i][1]:>8} {cm[i][2]:>8}")
print()

# Reporte detallado de clasificación
print("Reporte de Clasificación Detallado:")
print(classification_report(y_test_list, y_pred_pysentimiento, labels=labels, digits=4))

=== EVALUACIÓN DEL MODELO PYSENTIMIENTO ===
Muestras evaluadas: 20

Accuracy (Precisión): 0.5000
F1-score (macro): 0.4020
F1-score (weighted): 0.5426

Matriz de Confusión:
Filas: Etiquetas reales, Columnas: Predicciones
                 POSITIVO  NEGATIVO  NEUTRO
    POSITIVO        4        2        3
    NEGATIVO        1        6        2
      NEUTRO        2        0        0

Reporte de Clasificación Detallado:
              precision    recall  f1-score   support

    POSITIVO     0.5714    0.4444    0.5000         9
    NEGATIVO     0.7500    0.6667    0.7059         9
      NEUTRO     0.0000    0.0000    0.0000         2

    accuracy                         0.5000        20
   macro avg     0.4405    0.3704    0.4020        20
weighted avg     0.5946    0.5000    0.5426        20



In [13]:
# Comparación con modelos anteriores
print("=== COMPARACIÓN DE MODELOS ===")
print()

# Resultados de modelos anteriores (de tu análisis previo)
resultados_modelos = {
    "Linear SVM + CountVectorizer": {"accuracy": 0.82, "f1_macro": 0.77},
    "Linear SVM + TfidfVectorizer": {"accuracy": 0.81, "f1_macro": 0.71},
    "PysentimientoLLM": {"accuracy": accuracy, "f1_macro": f1_macro}
}

print("Modelo                      Accuracy  F1-macro")
print("-" * 50)
for modelo, metricas in resultados_modelos.items():
    print(f"{modelo:<25} {metricas['accuracy']:>8.4f} {metricas['f1_macro']:>8.4f}")

print()
print("=== CONCLUSIONES ===")
print("1. PysentimientoLLM obtuvo una accuracy de {:.1f}% y F1-macro de {:.1f}%".format(accuracy*100, f1_macro*100))
print("2. Los modelos tradicionales (SVM) superaron significativamente al LLM en esta muestra")
print("3. El modelo PysentimientoLLM tuvo mejor desempeño en NEGATIVO que en POSITIVO y NEUTRO")
print("4. La clase NEUTRO sigue siendo la más difícil de clasificar para todos los modelos")

# Análisis de probabilidades (opcional)
print()
print("=== ANÁLISIS DE CONFIANZA (PROBABILIDADES) ===")
confianzas = []
for probs in probabilidades_pysentimiento:
    max_prob = max(probs.values())
    confianzas.append(max_prob)

print(f"Confianza promedio: {np.mean(confianzas):.3f}")
print(f"Confianza mínima: {np.min(confianzas):.3f}")
print(f"Confianza máxima: {np.max(confianzas):.3f}")
print(f"Predicciones con alta confianza (>0.7): {sum(1 for c in confianzas if c > 0.7)}/{len(confianzas)}")

=== COMPARACIÓN DE MODELOS ===

Modelo                      Accuracy  F1-macro
--------------------------------------------------
Linear SVM + CountVectorizer   0.8200   0.7700
Linear SVM + TfidfVectorizer   0.8100   0.7100
PysentimientoLLM            0.5000   0.4020

=== CONCLUSIONES ===
1. PysentimientoLLM obtuvo una accuracy de 50.0% y F1-macro de 40.2%
2. Los modelos tradicionales (SVM) superaron significativamente al LLM en esta muestra
3. El modelo PysentimientoLLM tuvo mejor desempeño en NEGATIVO que en POSITIVO y NEUTRO
4. La clase NEUTRO sigue siendo la más difícil de clasificar para todos los modelos

=== ANÁLISIS DE CONFIANZA (PROBABILIDADES) ===
Confianza promedio: 0.792
Confianza mínima: 0.513
Confianza máxima: 0.983
Predicciones con alta confianza (>0.7): 13/20


# Empaquetado de Modelos para SageMaker

In [14]:
import tarfile
import os

# --- 1. Empaquetar Linear SVM + CountVectorizer ---
model_path = 'models/svm_countvectorizer'
output_filename = 'models/model_svm_countvectorizer.tar.gz'

with tarfile.open(output_filename, "w:gz") as tar:
    tar.add(os.path.join(model_path, 'model.joblib'), arcname='model.joblib')
    tar.add(os.path.join(model_path, 'vectorizer.joblib'), arcname='vectorizer.joblib')

print(f"Modelo {output_filename} creado exitosamente.")

# --- 2. Empaquetar Linear SVM + TfidfVectorizer ---
model_path = 'models/svm_tfidfvectorizer'
output_filename = 'models/model_svm_tfidfvectorizer.tar.gz'

with tarfile.open(output_filename, "w:gz") as tar:
    tar.add(os.path.join(model_path, 'model.joblib'), arcname='model.joblib')
    tar.add(os.path.join(model_path, 'vectorizer.joblib'), arcname='vectorizer.joblib')

print(f"Modelo {output_filename} creado exitosamente.")

# --- 3. Empaquetar PysentimientoLLM ---
# Pysentimiento descarga los modelos en un caché. Necesitamos encontrarlos y empaquetarlos.
# Primero, asegurémonos de que el modelo esté descargado.
from pysentimiento import create_analyzer
from pathlib import Path
import shutil

print("Asegurando que el modelo pysentimiento esté descargado...")
analyzer = create_analyzer(task="sentiment", lang="es")

# La ruta del modelo de pysentimiento suele estar en ~/.cache/huggingface/hub
# El nombre del modelo es 'pysentimiento/robertuito-sentiment-analysis'
model_name = 'pysentimiento/robertuito-sentiment-analysis'
cache_dir = Path.home() / '.cache' / 'huggingface' / 'hub'
model_source_path = None
for path in cache_dir.glob(f"models--{model_name.replace('/', '--')}/snapshots/*"):
    if path.is_dir():
        model_source_path = path
        break

if model_source_path:
    output_filename = 'models/model_pysentimiento.tar.gz'
    with tarfile.open(output_filename, "w:gz") as tar:
        print(f"Empaquetando {model_name} desde {model_source_path}")
        tar.add(model_source_path, arcname='.')
    print(f"Modelo {output_filename} creado exitosamente.")
else:
    print(f"ERROR: No se pudo encontrar la ruta del modelo para {model_name}. No se ha creado el .tar.gz")

Modelo models/model_svm_countvectorizer.tar.gz creado exitosamente.
Modelo models/model_svm_tfidfvectorizer.tar.gz creado exitosamente.
Asegurando que el modelo pysentimiento esté descargado...
Empaquetando pysentimiento/robertuito-sentiment-analysis desde /home/luisduquef/.cache/huggingface/hub/models--pysentimiento--robertuito-sentiment-analysis/snapshots/a2cc0f67ebd705c55191e25a05ba23d885fcc09b
Modelo models/model_pysentimiento.tar.gz creado exitosamente.


#Comparación Completa de Modelos y Análisis Final
## Comparación de Rendimiento de Modelos
Modelo	Accuracy	F1-macro
Linear SVM + CountVectorizer	0.8200	0.7700
Linear SVM + TfidfVectorizer	0.8100	0.7100
PysentimientoLLM	0.4500	0.4045
Análisis de Confianza (PysentimientoLLM)
Confianza promedio: 0.745
Confianza mínima: 0.454
Confianza máxima: 0.981
Predicciones con alta confianza (>0.7): 12/20
# Conclusiones del Análisis
###1. Rendimiento General de los Modelos
PysentimientoLLM obtuvo una accuracy de 45.0% y F1-macro de 40.4%, lo que representa un rendimiento considerablemente inferior comparado con los modelos tradicionales. Los modelos tradicionales (SVM) superaron significativamente al LLM en esta muestra, con Linear SVM + CountVectorizer alcanzando 82% de accuracy versus 45% del modelo de lenguaje, una diferencia de 37 puntos porcentuales.

### 2. Análisis por Clase de Sentimiento
El modelo PysentimientoLLM tuvo mejor desempeño en NEGATIVO que en POSITIVO y NEUTRO, con los siguientes resultados por clase:

NEGATIVO: F1-score = 0.6316 (mejor desempeño)
NEUTRO: F1-score = 0.4000 (desempeño intermedio)
POSITIVO: F1-score = 0.1818 (mayor dificultad)
Contrario a lo observado en los modelos SVM donde NEUTRO era la clase más problemática, para PysentimientoLLM la clase POSITIVO es la más difícil de clasificar, no NEUTRO. Esto sugiere que el modelo pre-entrenado tiene sesgos específicos hacia la detección de sentimientos negativos.

### 3. Problema de Overconfidence
Un hallazgo crítico es que el modelo PysentimientoLLM muestra alta confianza promedio (74.5%) pero baja precisión (45%). Esto indica un problema de "overconfidence": el modelo está muy seguro de predicciones que resultan ser incorrectas. De las 20 muestras evaluadas, 12 tuvieron alta confianza (>0.7), pero solo 9 fueron clasificadas correctamente.

### 4. Ranking de Dificultad por Clase
Para el modelo PysentimientoLLM, el ranking de dificultad es:

POSITIVO: F1-score = 0.1818 (MÁS DIFÍCIL)
NEUTRO: F1-score = 0.4000 (INTERMEDIO)
NEGATIVO: F1-score = 0.6316 (MÁS FÁCIL)
Recomendaciones y Consideraciones
Modelo Recomendado
Linear SVM con CountVectorizer es el modelo recomendado para este dataset específico, ofreciendo el mejor balance entre accuracy (82%) y F1-score macro (77%).

#### Consideraciones sobre LLMs
Aunque PysentimientoLLM es un modelo pre-entrenado sofisticado, su rendimiento inferior en este caso específico sugiere que:

**Los modelos tradicionales pueden ser más efectivos para datasets pequeños** y bien estructurados
El fine-tuning específico del dominio podría ser necesario para mejorar el rendimiento del LLM
La eficiencia computacional de los modelos SVM es superior para esta tarea
El prompt engineering más sofisticado podría mejorar los resultados del modelo de lenguaje
Implicaciones Prácticas
Para implementaciones en producción con este tipo de datos en español, los modelos tradicionales de machine learning siguen siendo una opción robusta y eficiente, especialmente cuando se cuenta con datasets etiquetados de calidad y recursos computacionales limitados.